In [ ]:
"""
README
William Ian McKinely
MSBI 31600 Advanced Project Winter 2024

1. Description of the project
This project is an initial framework for a pipeline which can take in a number of audio files which will be recordings from 
operating rooms collected for a research project examining teacher-learner verbal interactions. This uses the AWS Transcribe 
Medical service to transcribe the audio files into a JSON file, which can be pulled from AWS
servers and parsed into a formatted transcript which can be collected as a dataframe that can be passed to a downstream
analysis. 

2. Why I selected the project
I am the PI for this study, and I value the ability to plan for a long-term functional database of transcripts for analysis.
I think that we will begin to identify verbal patterns that we currently don't describe and which are unique to surgery, so
having something with a longstanding structure will be important when we need to include something retroactively. Also I'm
secretly super skeptical of the ability for AI to outperform humans in transcription, especially in our very specific use case
that is probably otherwise not well represented in the training data, so I was curious to explore it. Currently, the project
suffers from a limitation caused by the upper limits of capacity of human transcriptionists, which is compounded by the cost;
using AI would be a lot cheaper, but maybe not better in the long run.

I have learned a lot about the need for incredibly clean data in order for use of AI to be successful. Our audio files 
(the research files, which are not provided here) are real-world audio; they contain overlap, background noise, and 
other elements which easily confuse the software. To be fair, they often also confuse humans, but the humans know to 
notice it and find clarification or otherwise adjsut. I also have a much greater appreciation for anyone who makes something
that passes between systems because it is quite challenging. We will need to do a lot of work before this will be a viable
solution to our problem, but it was enough to get me interested, and I have a couple of ideas about where to go next 
(two channel audio input, custom vocabulary). 

3. Themes selected
Theme 1: JSON
I used JSON.loads() to get the data from a stream into a local variable; I also broke it in about every way you can imagine,
which is not shown here because I care about your happiness. I used the structure of the JSON to my advantage to reliably
parse it to a dataframe representing my desired format. Choosing this was almost a default choice; the organization structure
means I will almost certainly have a lot of my future occupied in the JSON format, so it would seem crazy to have possibly 
made a different choice.
Theme 2: Server/client relationships & APIs
I interfaced with the AWS API using the boko3 package. I selected this theme because I plan to do a lot of work with large
data sets in the future; this will inevitably mean use of a cloud service or remote HPC, and either will function along
similar lines so mastering navigation of this relationship is prudent. These elements are applied in the class declaration
for 'server_grab' because this is what connects to the AWS servers, starts the transcription job, and then pulls the data
in through a data stream.

4. What I would do differently
First, I would have liked to make this a much more in-depth exploration of the elements. However, limitations stemming
from both real life and the nature of writing code meant that this wasn't always possible or realistic. I would have really
liked to make a UI that allows the user to pick the files to be transcribed (currently you have to either manually list them,
have a folder for them which gets selected and then you go move them out manually later, or something like that) so that
more of the recordings could be stored in a single location but not all be transcribed repeatedly. I would also probably
have started versioning earlier, so that I could build a framework and piece it together. Instead, I have a smattering
of files which represent me going through the pieces and then a master where I eventually put them together. 

5. How to run the project
***

6. Challenges & Obstacles
This project was definitely challenging in the way I expected. I knew that I fundamentally would be able to make the JSON 
parsing work out, but it wasn't always easy to get the time modules to cooperate. I really did not enjoy engaging the AWS
API, and I found it very confusing to start but I eventually found my footing. I was able to answer a lot of questions for
myself about what is realistically possible, and was also able to determine which elements we will still need to work to 
overcome in the long term that can't be fixed programatically (like having 2 channel audio input, for example).

7. Sources
Sources are cited in-line throughout the code wherever they were used. I also used the AWS documentation for the Transcribe
Medical service to get me moving. 

8. Extra credit
***
I met the criteria for using try/except in the JSON parsing cell while setting the first starttime.
"""

In [1]:
import json
import datetime
from dataclasses import dataclass
import numpy as np
import pandas as pd
import boto3

In [63]:
j = dict()

@dataclass
class server_grab():

    full_data = pd.DataFrame(columns=['spk','start_time','end_time','transcript'])

    def __init__(self):
        self.spk:str = "spk_0"
        self.start_time = datetime.timedelta(days=0,seconds=0,milliseconds=0)
        self.end_time = datetime.timedelta(days=0,seconds=0,milliseconds=0)
        self.transcript:str = ""
        self.s3_client = boto3.client('s3')
        self.client = boto3.client('transcribe')
        self.s3_resource = boto3.resource('s3')
        bucket = self.s3_resource.Bucket(name='msbi31600')

    #Here we can submit the job to the transcription service
    #Future improvements would include dynamic file naming, selection for output locations, and multiple file handling
    #A better version would also add error handling for job submissions
    def sub_job(self):
        job = self.client.start_medical_transcription_job(
            MedicalTranscriptionJobName='test_job',
            LanguageCode='en-US',
            MediaFormat='m4a',
            Media={
                'MediaFileUri': 's3://msbi31600/test_audio.m4a'
            },
            OutputBucketName='msbi31600',
         #OutputKey='outputs/test_output2.json',
            Settings={
        'ShowSpeakerLabels': True,
        'MaxSpeakerLabels': 2,
        'ChannelIdentification': False
            },
            Specialty='PRIMARYCARE',
            Type = 'CONVERSATION'
        )
    
    #This function gets the transcription JSON file from the bucket and puts it into a variable for our use
    def get_json(self):
        data = self.s3_client.get_object(Bucket='msbi31600',Key='medical/test_job.json')
        body=data['Body']
        j = json.loads(body.read().decode("utf-8"))
        return j
        """
        results = self.s3_client.list_objects(Bucket='msbi31600', Prefix='medical/test_job')
        for file in results.get('Contents'):
            data = self.s3_client.get_object(Bucket='msbi31600', Key=file.get('Key'))
            contents = data['Body'].read()
            j = json.loads(contents,parse_float='Decimal')
            return j
        print(j)
        globals()[dvar] = j"""

    #This function deletes the job if you encounter the error that the job already exists
    #Future error handling might be able to catch this error and delete the job automatically, but it would have to be a precise situation because this could cause big problems
    def del_job(self):
        dummy = self.client.delete_medical_transcription_job(
            MedicalTranscriptionJobName='test_job'
        )


In [64]:
mine = server_grab()

In [11]:
#After you submit the job, you have to wait a while for the transcription to complete
#I need to add something to check the status of the job to enhance the user experience
mine.sub_job()

In [65]:
#This is a functional solution to getting the file out, but it doesn't play in our current OOP framework
#Need to figure out if we can make it work, or it we should instead just make a second function to get the file out
file = mine.get_json()

In [66]:
#This cell currently represents a functional parsing mechanism

start_time = datetime.timedelta(days=0,seconds=0,milliseconds=0)
end_time = datetime.timedelta(days=0,seconds=0,milliseconds=0)

full_data = pd.DataFrame(columns=['spk','start_time','end_time','transcript'])

second_start=int()
minute_start=int()
second_end=int()
minute_end=int()
spk = "spk_0"
transcript = ""

#This segment is the broadest loop, which is pulling each 'item' from the Amazon-produced JSON file and parsing it to the useful chunks we need
for item in file['results']['items']:
    
    #This block checks if the speaker label has changed at its outermost scope, and behaves differently depending on whether or not the speaker has changed
    #We may need to change the behavior at this level when we start using multi channel input in the future
    if item['speaker_label'] == spk:
        
        #First we have to see if the item is a punctuation mark, and if so, we need to add it to the transcript without a space and without changing the timestamp
        if item['type'] == 'punctuation':
            transcript = transcript + item['alternatives'][0]['content']
        elif item['type'] == 'pronunciation':

            #If the speaker hasn't changed, we add the content to the transcript
            #I plan to come back here later and add a checking function, where if the item type is 'punctuation' it will omit the added space before addending the content
            transcript = transcript + " " + item['alternatives'][0]['content']
        
            while True:
                try:
                    #We also check to see if the start time is listed as being 0.0 (i.e. if it has been reset) and if so, we set it to the first value we come across since the reset
                    #This allows us to set the correct start time for each line of text per speaker, which will be important for assigning pauses in the future
                    if start_time.total_seconds() == 0.0:
                        split_start = item['start_time'].split('.')
                        second_start = int(split_start[0])
                        millisecond_start = (split_start[1])
                        if len(millisecond_start) == 1:
                            millisecond_start = int(millisecond_start) * 100
                        elif len(millisecond_start) == 2:
                            millisecond_start = int(millisecond_start) * 10
                        else:
                            millisecond_start = int(millisecond_start)
                        start_time_hold = datetime.timedelta(days=0,seconds=second_start,milliseconds=millisecond_start)
                
                        #Lifted this directly from https://stackoverflow.com/a/539360 & modified it slightly for own use
                        s1= start_time_hold.total_seconds()
                        hours1, remain1 = divmod(s1, 3600)
                        minutes1, remain2_1 = divmod(remain1, 60)
                        #Remember that divisor on the following line is 1, because the whole thing is indexed to seconds
                        seconds1, remain3_1 = divmod(remain2_1, 1)
                        #We added this modifier to increase the remainder so the milliseconds format correctly
                        remain3_1 = remain3_1 * 1000
                        var1 = ('{:02}:{:02}:{:02}:{:03}'.format(int(hours1), int(minutes1), int(seconds1), int(remain3_1)))

                        start_time = var1
                        break
                    else:
                        break
                except AttributeError:
                    break
        
            #This block takes the end time from the JSON file and parses it into a timedelta object by splitting it and then putting it back together in the desired format
            split_end = item['end_time'].split('.')
            second_end = int(split_end[0])
            millisecond_end = (split_end[1])
            if len(millisecond_end) == 1:
                millisecond_end = int(millisecond_end) * 100
            elif len(millisecond_end) == 2:
                millisecond_end = int(millisecond_end) * 10
            else:
                millisecond_end = int(millisecond_end)
            end_time_hold = datetime.timedelta(days=0,seconds=second_end,milliseconds=millisecond_end)
            
            #Lifted this directly from https://stackoverflow.com/a/539360 & modified it slightly for own use
            s2= end_time_hold.total_seconds()
            hours2, remain2 = divmod(s2, 3600)
            minutes2, remain2_2 = divmod(remain2, 60)
            #Remember that divisor on the following line is 1, because the whole thing is indexed to seconds
            seconds2, remain3_2 = divmod(remain2_2, 1)
            #We added this modifier to increase the remainder so the milliseconds format correctly
            remain3_2 = remain3_2 * 1000
            var2 = ('{:02}:{:02}:{:02}:{:03}'.format(int(hours2), int(minutes2), int(seconds2), int(remain3_2)))
            
            #Then we replace the end time with the holder, to be sure it is updated every time and no failures will occur
            end_time = var2
    
    #This is the block which is called if the speaker changes
    elif item['speaker_label'] != spk:
        
        #We add the data currently in our containers to the dataframe
        full_data = full_data.append({'spk':spk,'start_time':start_time,'end_time':end_time,'transcript':transcript},ignore_index=True)
        
        #Change the speaker label to the new speaker
        spk = item['speaker_label']
        
        #Reset the start time to the current value
        split_start = item['start_time'].split('.')
        second_start = int(split_start[0])
        millisecond_start = (split_start[1])
        if len(millisecond_start) == 1:
            millisecond_start = int(millisecond_start) * 100
        elif len(millisecond_start) == 2:
            millisecond_start = int(millisecond_start) * 10
        else:
            millisecond_start = int(millisecond_start)
        start_time_hold = datetime.timedelta(days=0,seconds=second_start,milliseconds=millisecond_start)
        
        #Lifted this directly from https://stackoverflow.com/a/539360 & modified it slightly for own use
        s3= start_time_hold.total_seconds()
        hours3, remain3 = divmod(s3, 3600)
        minutes3, remain2_3 = divmod(remain3, 60)
        #Remember that divisor on the following line is 1, because the whole thing is indexed to seconds
        seconds3, remain3_3 = divmod(remain2_3, 1)
        #We added this modifier to increase the remainder so the milliseconds format correctly
        remain3_3 = remain3_3 * 1000
        var3 = ('{:02}:{:02}:{:02}:{:03}'.format(int(hours3), int(minutes3), int(seconds3), int(remain3_3)))
        start_time = var3

        #This block takes the end time from the JSON file and parses it into a timedelta object by splitting it and then putting it back together in the desired format
        split_end = item['end_time'].split('.')
        second_end = int(split_end[0])
        millisecond_end = (split_end[1])
        if len(millisecond_end) == 1:
            millisecond_end = int(millisecond_end) * 100
        elif len(millisecond_end) == 2:
            millisecond_end = int(millisecond_end) * 10
        else:
            millisecond_end = int(millisecond_end)
        end_time_hold = datetime.timedelta(days=0,seconds=second_end,milliseconds=millisecond_end)
        #Lifted this directly from https://stackoverflow.com/a/539360 & modified it slightly for own use
        sx= end_time_hold.total_seconds()
        hoursx, remainx = divmod(sx, 3600)
        minutesx, remainx2 = divmod(remainx, 60)
        #Remember that divisor on the following line is 1, because the whole thing is indexed to seconds
        secondsx, remainx3 = divmod(remainx2, 1)
        #We added this modifier to increase the remainder so the milliseconds format correctly
        remainx3 = remainx3 * 1000
        var4 = ('{:02}:{:02}:{:02}:{:03}'.format(int(hoursx), int(minutesx), int(secondsx), int(remainx3)))
        end_time = var4

        #Reset the transcript to be blank
        transcript = ""
        #Then add the current content to the transcript
        transcript = transcript + " " + item['alternatives'][0]['content']

#When the whole thing has run, the final line of data will not have been added to the dataframe, so we add it here
full_data = full_data.append({'spk':spk,'start_time':start_time,'end_time':end_time,'transcript':transcript},ignore_index=True)



{'jobName': 'test_job',
 'accountId': '211125774148',
 'status': 'COMPLETED',
 'results': {'transcripts': [{'transcript': "This is gonna be a silent teacher. It's the way to do it. Experiential learning more college now. It's a microphone. It's for study. We're doing these, I've decided to record the surgeons in all my cases for evidence, evidentiary reasons. All of us are working. I just lost. Right. OK. No, so. Right. Thank you. That. Yeah. Yeah. Yeah. Yeah. Keep all that fat up. Yeah. Look at the other side. Make sure you know cause it's gonna go flat now. Right? It's going straight down so need to be gone that. Yeah. It uh Right. Right. Got it. Did I get the camera cleans? Yeah. Something they got you. Yep. Good. Thank you. No, sorry the it um thank you. I going to be gone more immediately. Right. Yeah, that. No, my name is my name. I'm sorry. Too late for that now. I think I Yeah, I had a baby. Yeah, I mean Alex didn't care either. He's like whatever you want. He actually was. Yea